In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Montecarlo calculation of π
%matplotlib inline

In [3]:
def mc_pi(ntrials):
    """
    Calculate the value of pi using montecarlo method and visualize the process
    """
    x = np.random.random(ntrials)
    y = np.random.random(ntrials)
    #masking
    inside_circle = x**2+y**2 < 1
    unit_circle_x = np.linspace(0,1,100)
    unit_circle = [unit_circle_x,np.sqrt(1.0-unit_circle_x**2)]
    plt.plot(*unit_circle,color ='black')
    plt.scatter(x[inside_circle],y[inside_circle],marker='.',color='blue',s=1)
    plt.scatter(x[~inside_circle],y[~inside_circle],marker='.',color='red',s=1)
    plt.title("value of $\pi$="+str(4.0*np.sum(inside_circle)/float(ntrials)))

In [4]:
from ipywidgets import interact, interactive, interact_manual
mc_widget = interactive(mc_pi,ntrials=(1,100000,10));
mc_widget

interactive(children=(IntSlider(value=49991, description='ntrials', max=100000, min=1, step=10), Output()), _d…

In [5]:
#SVD for image compression
from skimage import data
from skimage.color import rgb2gray

In [6]:
from skimage import img_as_ubyte,img_as_float
gray_images = {
    "cat":rgb2gray(img_as_float(data.chelsea())),
    "astro":rgb2gray(img_as_float(data.astronaut())),
    "camera":data.camera(),
    "coin": data.coins(),
    "clock":data.clock(),
    "blobs":data.binary_blobs(),
    "coffee":rgb2gray(img_as_float(data.coffee()))
}

In [7]:
from numpy.linalg import svd

In [ ]:
###################################################################################################

In [8]:
def compress_svd(image, k):
    """
    Perform svd decomposition and truncated(using k singular values/vectors) reconstruction
    returns
    ------
    reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,v = svd(image,full_matrices = False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),v[:k,:]))
    
    return reconst_matrix,s

In [9]:
def compress_show_gray_images(img_name,k):
    """
    Compresses gray scale images and display the reconstructed image.
    Also displays a plot of singular values
    """
    image=gray_images[img_name]
    original_shape = image.shape
    reconst_img,s=compress_svd(image,k)
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    axes[0].plot(s)
    compression_ratio = 100.0*(k*(original_shape[0] +
                                  original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("compression ratio= {:,2f}"+format(compression_ratio)+"%")
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('on')
    fig.tight_layout() 

In [10]:
interact(compress_show_gray_images,img_name=list(gray_images.keys()),k=(1,300));

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro', 'camera', 'coin', 'clock', 'bl…

In [11]:
def compress_show_gray_images(img_name,k,original_shape):
    """
    Compresses gray scale images and display the reconstructed image.
    Also displays a plot of singular values
    """
    image=gray_images[img_name]
    original_shape = image.shape
    reconst_img,s=compress_svd(image,k)
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    axes[0].plot(s)
    #K is the number of singular values/vectors
    compression_ratio = 100.0*(k*(original_shape[0] +
                                  original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("compression ratio= {:,2f}"+format(compression_ratio)+"%")
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('on')
    fig.tight_layout() 

In [12]:
color_images = {
    "cat":img_as_float(data.chelsea()),
    "astro":img_as_float(data.astronaut()),
    "coffee":img_as_float(data.coffee())
}

In [13]:
#Reshaping method to compress a color image involves flattening the third dimension of the 
#image array into the second dimension using numpy's reshape method.
def compress_show_color_images_reshape(img_name,k):
    """
    compress and display the reconstructed color image using the reshape method
    """
    image = color_images[img_name]
    original_shape = image.shape
    image_reshaped = image.reshape((original_shape[0],original_shape[1]*3))
    image_reconst,_ =compress_svd(image_reshaped,k)
    image_reconst = image_reconst.reshape(original_shape)
    compression_ratio = 100.0*(k*(original_shape[0] +
                                  3*original_shape[1])+k)/(original_shape[0]*original_shape[1]*original_shape[2])
    plt.title("compression ratio= {:,2f}"+format(compression_ratio)+"%")
    plt.imshow(image_reconst)

In [14]:
interact(compress_show_color_images_reshape,img_name = list(color_images.keys()),k=(0,512));

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro', 'coffee'), value='cat'), IntSl…

In [15]:
#In this layers method to compress color images we treat a color image as a stack
#of 3 seperate two dimensional images (RGB). we apply the truncated svd reconstruction
#on each two dimensional layer seperately.
def compress_show_color_images_layer(img_name,k):
    """
    compress and display the reconstructed color image using the reshape method
    """
    image = color_images[img_name]
    original_shape = image.shape
    image_reconst_layers = [compress_svd(image[:,:,i],k)[0] for i in range(3)]
    image_reconst = np.zeros(image.shape)
    for i in range(3):
        image_reconst[:,:,i] = image_reconst_layers[i]
        compression_ratio = 100.0*(k*(original_shape[0] +
                                  3*original_shape[1])+k)/(original_shape[0]*original_shape[1]*original_shape[2])
        plt.title("compression ratio= {:,2f}"+format(compression_ratio)+"%")
        plt.imshow(image_reconst)

In [17]:
interact(compress_show_color_images_layer,img_name=list(color_images.keys()),k=(1,550));

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro', 'coffee'), value='cat'), IntSl…